In [25]:
!python -V

Python 3.12.2


In [26]:
import pandas as pd

In [27]:
import pickle

In [28]:

import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='file:///c:/Users/henri/Desktop/Ideias/MLOps/zoomcamp/mlops-zoomcamp/mlruns/1', creation_time=1754160086452, experiment_id='1', last_update_time=1754160086452, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [29]:
import seaborn as sns
import matplotlib.pyplot as plt

In [30]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [31]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [32]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [33]:
len(df_train), len(df_val)

(73908, 61921)

In [34]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [35]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [36]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [37]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715210601492

In [38]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [40]:
with mlflow.start_run():
    mlflow.set_tag("developer", "henrique")
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
    mlflow.log_param("model", "Lasso")
    mlflow.log_param("random_state", 42)

    alpha = 0.1
    mlflow.log_param("alpha",alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    print(f"rmse: {rmse}")

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")



rmse: 12.143233939842847


In [20]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params = params, 
            dtrain = train,
            num_boost_round = 100,
            evals=[(valid, "validation")],
            early_stopping_rounds=50,
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        return {"loss": rmse, "status": STATUS_OK}



In [24]:
search_space = {
    "max_depth":scope.int(hp.quniform("max_depth", 4, 10, 1)),
    "learning_rate":hp.loguniform("learning_rate", -3, 0),
    "reg_alpha":hp.loguniform("reg_alpha", -5, -1),
    "reg_lambda":hp.loguniform("reg_lambda", -6, 1),
    "min_child_weight":hp.loguniform("iiiii", -1, 3),
    "objective":"reg:linear",
    "seed":42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials(),
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Users\henri\Desktop\Ideias\MLOps\zoomcamp\mlops-zoomcamp\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [21:03:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.42929                           
[1]	validation-rmse:7.28204                           
[2]	validation-rmse:6.95485                           
[3]	validation-rmse:6.85412                           
[4]	validation-rmse:6.80960                           
[5]	validation-rmse:6.79313                           
[6]	validation-rmse:6.78451                           
[7]	validation-rmse:6.77873                           
[8]	validation-rmse:6.77610                           
[9]	validation-rmse:6.76392                           
[10]	validation-rmse:6.76185                          
[11]	validation-rmse:6.75731                          
[12]	validation-rmse:6.75680                          
[13]	validation-rmse:6.75273                          
[14]	validation-rmse:6.75098                          
[15]	validation-rmse:6.74444                          
[16]	validation-rmse:6.74000                          
[17]	validation-rmse:6.73799                          
[18]	valid

c:\Users\henri\Desktop\Ideias\MLOps\zoomcamp\mlops-zoomcamp\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [21:03:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:10.49596                                                   
[1]	validation-rmse:9.28355                                                    
[2]	validation-rmse:8.44763                                                    
  2%|▏         | 1/50 [00:07<06:31,  7.99s/trial, best loss: 6.610004737509731]


KeyboardInterrupt: 

In [56]:
with mlflow.start_run():

    best_params = {
        "learning_rate": 0.7779257409652834,
        "max_depth": 10,
        "min_child_weight": 5.363379742991612,
        "objective": "reg:linear",
        "reg_alpha": 0.0072572893963142125,
        "reg_lambda": 0.24330664896106105,
        "seed": 42
    }

    mlflow.log_params(best_params)
    booster = xgb.train(params = best_params,
    dtrain = train,
    num_boost_round = 200,
    evals = [(valid, "validation")],
    early_stopping_rounds = 50)

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact("models/preprocessor.b", artifact_path = "preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path =  "models", registered_model_name = "xgboost")

c:\Users\henri\Desktop\Ideias\MLOps\zoomcamp\mlops-zoomcamp\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [19:36:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:7.26875
[1]	validation-rmse:6.79091
[2]	validation-rmse:6.71203
[3]	validation-rmse:6.69987
[4]	validation-rmse:6.68009
[5]	validation-rmse:6.67416
[6]	validation-rmse:6.67086
[7]	validation-rmse:6.66772
[8]	validation-rmse:6.66102
[9]	validation-rmse:6.65331
[10]	validation-rmse:6.64713
[11]	validation-rmse:6.63922
[12]	validation-rmse:6.63397
[13]	validation-rmse:6.62968
[14]	validation-rmse:6.62684
[15]	validation-rmse:6.61993
[16]	validation-rmse:6.61511
[17]	validation-rmse:6.61312
[18]	validation-rmse:6.60978
[19]	validation-rmse:6.60840
[20]	validation-rmse:6.60337
[21]	validation-rmse:6.60087
[22]	validation-rmse:6.59904
[23]	validation-rmse:6.59633
[24]	validation-rmse:6.59292
[25]	validation-rmse:6.58808
[26]	validation-rmse:6.58365
[27]	validation-rmse:6.58097
[28]	validation-rmse:6.57613
[29]	validation-rmse:6.57571
[30]	validation-rmse:6.57336
[31]	validation-rmse:6.57172
[32]	validation-rmse:6.56751
[33]	validation-rmse:6.56526
[34]	validation-rmse:6.5

2025/08/05 19:36:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\henri\Desktop\Ideias\MLOps\zoomcamp\mlops-zoomcamp\venv\Lib\site-packages\mlflow\xgboost\__init__.py:168: UserWarning: [19:36:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/08/05 19:36:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'xgboost' already exists. Creating a new version of this model...
Created version '2' of model 'xgboost'.


In [49]:
logged_model = "models:/m-6d15a27ea877480e8aa23d3625c81f08"
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model.predict(X_val)

array([15.280353 ,  7.278186 , 15.697888 , ..., 13.737298 ,  5.9582467,
        8.365794 ], shape=(61921,), dtype=float32)

In [53]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model.predict(valid)

array([15.280353 ,  7.278186 , 15.697888 , ..., 13.737298 ,  5.9582467,
        8.365794 ], shape=(61921,), dtype=float32)